In [1]:
import argparse
import os
import random

import numpy as np
import torch
import torch.nn as nn

from model_pytorch import LMModel, load_openai_pretrained_model
from text_utils import TextEncoder

In [16]:
n_vocab + n_special

40478

In [19]:
np.array(X).shape[-1]

13

In [2]:
def make_batch(X):
    X = np.array(X)
    assert X.ndim in [1, 2]
    if X.ndim == 1:
        X = np.expand_dims(X, axis=0)
    pos_enc = np.arange(n_vocab + n_special, n_vocab + n_special + X.shape[-1])
    pos_enc = np.expand_dims(pos_enc, axis=0)
    batch = np.stack([X, pos_enc], axis=-1)
    batch = torch.tensor(batch, dtype=torch.long).to(device)
    return batch

def append_batch(X, next_idx):
    next_pos = X[:, -1:, 1] + 1
    next_x = torch.cat((next_idx, next_pos), -1).unsqueeze(1)
    return torch.cat((X, next_x), 1)

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--desc', type=str, help="Description")
parser.add_argument('--dataset', type=str)
parser.add_argument('--log_dir', type=str, default='log/')
parser.add_argument('--save_dir', type=str, default='save/')
parser.add_argument('--data_dir', type=str, default='data/')
parser.add_argument('--submission_dir', type=str, default='submission/')
parser.add_argument('--submit', action='store_true')
parser.add_argument('--analysis', action='store_true')
parser.add_argument('--seed', type=int, default=42)
parser.add_argument('--n_iter', type=int, default=3)
parser.add_argument('--n_batch', type=int, default=8)
parser.add_argument('--max_grad_norm', type=int, default=1)
parser.add_argument('--lr', type=float, default=6.25e-5)
parser.add_argument('--lr_warmup', type=float, default=0.002)
parser.add_argument('--n_ctx', type=int, default=512)
parser.add_argument('--n_embd', type=int, default=768)
parser.add_argument('--n_head', type=int, default=12)
parser.add_argument('--n_layer', type=int, default=12)
parser.add_argument('--embd_pdrop', type=float, default=0.1)
parser.add_argument('--attn_pdrop', type=float, default=0.1)
parser.add_argument('--resid_pdrop', type=float, default=0.1)
parser.add_argument('--clf_pdrop', type=float, default=0.1)
parser.add_argument('--l2', type=float, default=0.01)
parser.add_argument('--vector_l2', action='store_true')
parser.add_argument('--opt', type=str, default='adam')
parser.add_argument('--afn', type=str, default='gelu')
parser.add_argument('--lr_schedule', type=str, default='warmup_linear')
parser.add_argument('--encoder_path', type=str, default='model/encoder_bpe_40000.json')
parser.add_argument('--bpe_path', type=str, default='model/vocab_40000.bpe')
parser.add_argument('--n_transfer', type=int, default=12)
parser.add_argument('--lm_coef', type=float, default=0.5)
parser.add_argument('--b1', type=float, default=0.9)
parser.add_argument('--b2', type=float, default=0.999)
parser.add_argument('--e', type=float, default=1e-8)
parser.add_argument('--n_valid', type=int, default=374)
parser.add_argument('--gen_len', type=int, default=20)
parser.add_argument('--topk', type=int, default=10)

args = parser.parse_args([])
print(args)

Namespace(afn='gelu', analysis=False, attn_pdrop=0.1, b1=0.9, b2=0.999, bpe_path='model/vocab_40000.bpe', clf_pdrop=0.1, data_dir='data/', dataset=None, desc=None, e=1e-08, embd_pdrop=0.1, encoder_path='model/encoder_bpe_40000.json', gen_len=20, l2=0.01, lm_coef=0.5, log_dir='log/', lr=6.25e-05, lr_schedule='warmup_linear', lr_warmup=0.002, max_grad_norm=1, n_batch=8, n_ctx=512, n_embd=768, n_head=12, n_iter=3, n_layer=12, n_transfer=12, n_valid=374, opt='adam', resid_pdrop=0.1, save_dir='save/', seed=42, submission_dir='submission/', submit=False, topk=10, vector_l2=False)


In [4]:
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

# Constants
submit = args.submit
dataset = args.dataset
n_ctx = args.n_ctx
save_dir = args.save_dir
desc = args.desc
data_dir = args.data_dir
log_dir = args.log_dir
submission_dir = args.submission_dir

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("device", device, "n_gpu", n_gpu)

text_encoder = TextEncoder(args.encoder_path, args.bpe_path)
encoder = text_encoder.encoder
n_vocab = len(text_encoder.encoder)

n_special = 0   # XD: useless for language modeling task
vocab = n_vocab + n_special + n_ctx

lm_model = LMModel(args, vocab, n_ctx, return_probs=True)
load_openai_pretrained_model(lm_model.transformer, n_ctx=n_ctx, n_special=n_special)
lm_model.to(device)

lm_model.eval()

device cuda n_gpu 1
Loading weights...


LMModel(
  (transformer): TransformerModel(
    (embed): Embedding(40990, 768)
    (drop): Dropout(p=0.1)
    (h): ModuleList(
      (0): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1)
          (resid_dropout): Dropout(p=0.1)
        )
        (ln_1): LayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1)
        )
        (ln_2): LayerNorm()
      )
      (1): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1)
          (resid_dropout): Dropout(p=0.1)
        )
        (ln_1): LayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1)
        )
        (ln_2): LayerNorm()
      )
      (2): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
      

In [13]:
# text = 'Hello, here is a sentence so I can test out what actually gets input into the linear model'
text = 'What the hell is this thing that we call positional encoding'
X = text_encoder.encode([text,])

In [14]:
XMB = make_batch(X)

In [15]:
XMB

tensor([[[  599, 40478],
         [  481, 40479],
         [ 1491, 40480],
         [  544, 40481],
         [  616, 40482],
         [  615, 40483],
         [  525, 40484],
         [  606, 40485],
         [ 1370, 40486],
         [ 3589, 40487],
         [ 1586, 40488],
         [  496, 40489],
         [33927, 40490]]], device='cuda:0')

In [11]:
XMB

tensor([[[ 3570, 40478],
         [  240, 40479],
         [  793, 40480],
         [  544, 40481],
         [  246, 40482],
         [ 5958, 40483],
         [  620, 40484],
         [  249, 40485],
         [  759, 40486],
         [ 2345, 40487],
         [  551, 40488],
         [  599, 40489],
         [ 1629, 40490],
         [ 2694, 40491],
         [19449, 40492],
         [  666, 40493],
         [  481, 40494],
         [35422, 40495],
         [ 7129, 40496]]], device='cuda:0')

In [5]:
text = input('Input some beginning words:')
while text != 'q':
    X = text_encoder.encode([text,])
    XMB = make_batch(X)

    for _ in range(args.gen_len):
        lm_probs = lm_model(XMB)
        if args.topk == 0:
            next_idx = torch.multinomial(lm_probs[:, -1, :], 1)
        else:
            values, indices = lm_probs[:, -1, :].topk(args.topk)
            next_idx = indices.gather(-1, torch.multinomial(values, 1))
        next_token = text_encoder.decoder[next_idx.item()].replace('</w>', '')
        print(next_token, end=' ')
        XMB = append_batch(XMB, next_idx)

    print()
    text = input('Input some beginning words:')

device cuda n_gpu 1
Loading weights...
Input some beginning words:Hello there


. it 's a long story . " 
 the woman sighed . " i can help you with that 
Input some beginning words:Wow this is pretty coolio


, ' said granny , who 'd got on to the old man 's back . ' how come you 
Input some beginning words:quit


. she did n't know how to react , but he seemed to be the most confident man that she 
Input some beginning words:ii


, the world is in chaos , the people are dying and the people are dying , and the only 


KeyboardInterrupt: 